## Depedencies Setup

In [ ]:
!pip install pandas

In [ ]:
!pip install openpyxl

In [ ]:
!pip install -U scikit-learn


In [ ]:
!pip install transformers


In [ ]:
!pip install datasets

In [ ]:
!pip install matplotlib

In [ ]:
!pip install torch

In [ ]:
!pip install accelerate -U


In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils import resample
from transformers import Trainer
from datasets import load_metric
import matplotlib.pyplot as plt
from datasets import load_dataset
import os
from transformers import AutoTokenizer, AutoModel
# import seaborn as sns
import pandas as pd
import numpy as np
from transformers import AutoModelForSequenceClassification

import preprocessor as p
import regex as re
import string

from datasets import Dataset



## PATH CONFIG SETUP

In [2]:
DATA = os.path.join(os.getcwd(),'data')
NLP = os.path.join(os.getcwd(),'NLP_bahasa_resources')


In [3]:
paths={
    'DATA_CLEAN': os.path.join(DATA,'feature.xlsx'),
    'TRAIN': os.path.join(DATA, 'train.csv'),
    'VALIDATION': os.path.join(DATA, 'eval.csv'),
    'TEST': os.path.join(DATA, 'test.csv'),
    'ROOT':os.path.join(NLP, 'combined_root_words.txt'),
    'SLANG': os.path.join(NLP, 'combined_slang_words.txt'),
    'STOPWORD': os.path.join(NLP, 'combined_stop_words.txt'),
    'SAVE_MODEL': os.path.join(os.getcwd(), 'model')
}


## DATASET SETUP

### Load Dataset

In [4]:
feature = pd.read_excel(paths['DATA_CLEAN'])


In [5]:
feature

,kategori_usaha,output_usaha,kode_kategori,kode_kelompok,text,punctuation,tokenization,stopwords,stemming
0,jual beli sarana upacara,banten dari janur,c,16292.0,jual beli sarana upacara banten dari janur,jual beli sarana upacara banten dari janur,"['jual', 'beli', 'sarana', 'upacara', 'banten'...","['jual', 'beli', 'sarana', 'upacara', 'banten'...","['jual', 'beli', 'sarana', 'upacara', 'banten'..."
1,menjual makanan ringan,makanan ringan,g,47112.0,menjual makanan ringan makanan ringan,menjual makanan ringan makanan ringan,"['menjual', 'makanan', 'ringan', 'makanan', 'r...","['menjual', 'makanan', 'ringan', 'makanan', 'r...","['jual', 'makan', 'ringan', 'makan', 'ringan']"
2,ukir sanggah dari kayu,ukiran kayu,c,16221.0,ukir sanggah dari kayu ukiran kayu,ukir sanggah dari kayu ukiran kayu,"['ukir', 'sanggah', 'dari', 'kayu', 'ukiran', ...","['ukir', 'sanggah', 'kayu', 'ukiran', 'kayu']","['ukir', 'sanggah', 'kayu', 'ukir', 'kayu']"
3,menjual makanan ringan,makanan ringan,g,47112.0,menjual makanan ringan makanan ringan,menjual makanan ringan makanan ringan,"['menjual', 'makanan', 'ringan', 'makanan', 'r...","['menjual', 'makanan', 'ringan', 'makanan', 'r...","['jual', 'makan', 'ringan', 'makan', 'ringan']"
4,jual minuman kopi,minuman kopi,i,56306.0,jual minuman kopi minuman kopi,jual minuman kopi minuman kopi,"['jual', 'minuman', 'kopi', 'minuman', 'kopi']","['jual', 'minuman', 'kopi', 'minuman', 'kopi']","['jual', 'minum', 'kopi', 'minum', 'kopi']"
...,...,...,...,...,...,...,...,...,...
394518,praktik doktor umum,jasa praktik doktor umum,q,86201.0,praktik doktor umum jasa praktik doktor umum,praktik doktor umum jasa praktik doktor umum,"['praktik', 'doktor', 'umum', 'jasa', 'praktik...","['praktik', 'doktor', 'jasa', 'praktik', 'dokt...","['praktik', 'doktor', 'jasa', 'praktik', 'dokt..."
394519,menjahit pakaian yang diinginkan,jasa menjahit pakaian,c,14120.0,menjahit pakaian yang diinginkan jasa menjahit...,menjahit pakaian yang diinginkan jasa menjahit...,"['menjahit', 'pakaian', 'yang', 'diinginkan', ...","['menjahit', 'pakaian', 'jasa', 'menjahit', 'p...","['jahit', 'pakai', 'jasa', 'jahit', 'pakai']"
394520,menjual gas epiji,gas elpiji,g,47772.0,menjual gas epiji gas elpiji,menjual gas epiji gas elpiji,"['menjual', 'gas', 'epiji', 'gas', 'elpiji']","['menjual', 'gas', 'epiji', 'gas', 'elpiji']","['jual', 'gas', 'epiji', 'gas', 'elpiji']"
394521,menjual eceran sembako di warung,makanan dan minuman,g,47112.0,menjual eceran sembako di warung makanan dan m...,menjual eceran sembako di warung makanan dan m...,"['menjual', 'eceran', 'sembako', 'di', 'warung...","['menjual', 'eceran', 'sembako', 'warung', 'ma...","['jual', 'ecer', 'sembako', 'warung', 'makan',..."


In [6]:
df = feature[['kategori_usaha','kode_kategori']]

In [7]:
df

,kategori_usaha,kode_kategori
0,jual beli sarana upacara,c
1,menjual makanan ringan,g
2,ukir sanggah dari kayu,c
3,menjual makanan ringan,g
4,jual minuman kopi,i
...,...,...
394518,praktik doktor umum,q
394519,menjahit pakaian yang diinginkan,c
394520,menjual gas epiji,g
394521,menjual eceran sembako di warung,g


In [8]:
df.rename(columns={'kategori_usaha':'text','kode_kategori':'label'},inplace = True)

C:\Users\adiag\AppData\Local\Temp\ipykernel_4352\2006725409.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'kategori_usaha':'text','kode_kategori':'label'},inplace = True)


In [9]:
df

,text,label
0,jual beli sarana upacara,c
1,menjual makanan ringan,g
2,ukir sanggah dari kayu,c
3,menjual makanan ringan,g
4,jual minuman kopi,i
...,...,...
394518,praktik doktor umum,q
394519,menjahit pakaian yang diinginkan,c
394520,menjual gas epiji,g
394521,menjual eceran sembako di warung,g


### Cleaning dataset

In [10]:
stop_words = open(paths['STOPWORD']).read().splitlines()


In [11]:
root_words = open(paths['ROOT']).read().splitlines()


In [12]:
def cleansing(text):   
    # Mengubah setiap kata menjadi lowercase
    text =  str(text).lower() 
    # Menghapus Link Dengan Pattern http/https dan www
    text = re.sub(r'http\S+', '', text)
    text = re.sub('(@\w+|#\w+)', '', text)
    # Menghapus Tag HTML
    text = re.sub('<.*?>', '', text)
    # Menghapus Tanda Baca Seperti Titik Dan Koma
    text = text.translate(str.maketrans(' ', ' ', string.punctuation))
    # Menghapus Karakter Selain Huruf a-z dan A-Z
    text = re.sub('[^a-zA-Z]', ' ', text)
    # Mengganti baris baru (enter) dengan spasi
    text = re.sub("\n", " ", text)
    # Menghapus Karakter Berulang (Contoh: Ya!!! menjadi Ya!)
    text = re.sub(r'(\w)(\1{2,})', r"\1", text)
    # Menghapus 1 Karakter Terpisah
    text = re.sub(r"\b[a-zA-Z]\b", "", text)
    # Menghapus Spasi Yang Lebih Dari Satu
    text = re.sub('(s{2,})', ' ', text)
    # Menghapus kata yang mengandung judul topik dan kata yang terdapat pada stopwords indonesia
    temp_text_split = []
    text_split = text.split(' ')

    for i in range(len(text_split)):
        if text_split[i] not in stop_words:
            # if text_split[i] in root_words:
            #      text_split[i] = root_words[text_split[i]]

            temp_text_split.append(str(text_split[i]))
    
    temp_text_split = list(set(temp_text_split))  
    text = ' '.join(temp_text_split)
       
    # Mengembalikan Hasil Preprocessing Text
    return str.lstrip(text)

In [13]:
df['text']

0                      jual beli sarana upacara
1                        menjual makanan ringan
2                        ukir sanggah dari kayu
3                        menjual makanan ringan
4                             jual minuman kopi
                          ...                  
394518                      praktik doktor umum
394519         menjahit pakaian yang diinginkan
394520                        menjual gas epiji
394521         menjual eceran sembako di warung
394522    menyablon baju sesuai yang diinginkan
Name: text, Length: 394523, dtype: object

In [14]:
df['text'] = df['text'].apply(cleansing)
df = df[df['text'].str.split().str.len().gt(2)]

print("Total Dataset Latih :", len(df))


C:\Users\adiag\AppData\Local\Temp\ipykernel_4352\1477846019.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(cleansing)


Total Dataset Latih : 260401


In [15]:
df

,text,label
0,beli sarana upacara jual,c
1,menjual makanan ringan,g
2,sanggah ukir kayu,c
3,menjual makanan ringan,g
4,kopi minuman jual,i
...,...,...
394515,masakan menjual makanan membuat olahan,c
394516,menjual keliling pisang membuat goreng,c
394517,menerima mentah barang mengolah jual,c
394520,gas menjual epiji,g


In [16]:
kode_n = df[df["label"] == "n"]
kode_p = df[df["label"] == "p"]
kode_j = df[df["label"] == "k"]


In [19]:
kode_j


,text,label
810,pinjam sindan jasa,k
1120,simpan pinjam jasa,k
1157,aktivitas simpan pinjam,k
1158,aktivitas simpan uang pinjam,k
1293,gadai kredit jasa,k
...,...,...
394375,simpan melayani pinjam,k
394388,simpan koperasi pinjam,k
394406,simpan koperasi pinjam,k
394417,simpan koperasi pinjam,k


In [18]:
kode_p

,text,label
467,asyidikiyah paud pendidikan jasa,p
473,sekolah dasar pendidikan jasa,p
522,kanak taman pendidikan,p
523,pedidikan menengah sekolah jasa,p
1171,kanak menyelenggarakan pendidikan,p
...,...,...
393672,dasar pendidikan sekolah,p
394326,perguruan kanda empat,p
394420,anak usia pendidikan,p
394429,menyediakan bimbingan belajar jasa,p


In [17]:
kode_n

,text,label
274,alat olah menyewakan raga,n
441,tour penjualan malam agen tiket paket bus,n
601,alat persewaan perkawinan,n
1099,permainan game menyewakan ps,n
1139,kertas fotocopy jasa,n
...,...,...
394043,copy atk menjual foto,n
394341,mobil travel paket menyewakan,n
394426,kerja manajemen penyediaan tenaga,n
394452,game menyewakan ps,n


In [ ]:
n_downsample = resample(kode_n,
                        replace=True,
                        n_samples=500,
                        random_state=42)

print(n_downsample.shape)


In [ ]:
p_downsample = resample(kode_p,
                        replace=True,
                        n_samples=500,
                        random_state=42)

print(p_downsample.shape)


In [ ]:
j_downsample = resample(kode_j,
                        replace=True,
                        n_samples=500,
                        random_state=42)

print(n_downsample.shape)


In [ ]:
df_clean = pd.concat([n_downsample, p_downsample, j_downsample])


In [ ]:
df_clean['label'].value_counts()

In [ ]:
df_clean

In [ ]:
df_clean["label"] = df_clean["label"].map({"n": 0, "p": 1, 'k': 2})


In [ ]:
df_clean

In [ ]:
def train_val_test_split(df):
    RANDOM_SEED = 42

    df_train, df_test = train_test_split(
        df,
        test_size=0.1,
        random_state=RANDOM_SEED,
    )

    df_val, df_test = train_test_split(
        df_test,
        test_size=0.5,
        random_state=RANDOM_SEED,
    )


In [ ]:
df_train.shape

In [ ]:
actual_label = df_test['label']


In [ ]:
actual_label.shape

In [ ]:
paths['TRAIN']

In [ ]:
df_train.to_csv(paths['TRAIN'], index=False)
df_val.to_csv(paths['VALIDATION'], index=False)
df_test.to_csv(paths['TEST'], index=False)


In [ ]:
files = {
        "train": paths['TRAIN'], 
        "eval": paths['VALIDATION'], 
        "test": paths['TEST']
    }

dataset = load_dataset('csv', data_files=files)

In [ ]:
dataset

## Model Tokenizer and Classification

### Tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p2")


In [ ]:
def tokenize_function(text):
    return tokenizer(text["text"], padding='max_length', max_length=256)

tokenized_datasets = dataset.map(tokenize_function, batched=True)


In [ ]:
tokenized_datasets


In [ ]:
train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["eval"]


In [ ]:
train_dataset['feature']

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "indobenchmark/indobert-base-p2", num_labels=3)


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer")


In [ ]:
metric = load_metric("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    return metric.compute(predictions=predictions, references=labels)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

train_history = trainer.train()
train_evaluate = trainer.evaluate()


In [ ]:
trainer


In [ ]:
test_dataset = tokenized_datasets["test"]

prediction = trainer.predict(test_dataset)
prediction = prediction.predictions.argmax(1)

prediction


In [ ]:
test_dataset = tokenized_datasets["test"]


In [ ]:
testing = trainer.predict(token_test)

In [ ]:
testing = testing.predictions.argmax(1)

In [ ]:
testing

In [ ]:
os.path.join(os.getcwd(),'model')

In [ ]:
trainer.save_model(paths['SAVE_MODEL'])


In [ ]:
tokenizer.save_pretrained(paths['SAVE_MODEL'])


In [ ]:
def getUserInput(text):
    ds = Dataset.from_dict({"text": [text]})
    return ds


In [ ]:
def getPredict(raw_text):
    text = cleansing(raw_text)
    dataset = getUserInput(text)
    token = dataset.map(tokenize_function, batched=True)
    predict = trainer.predict(token)
    predict = predict.predictions.argmax(1)
    predict = predict[0]
    if predict == 0:
        label = "N"
        desc = "Aktivitas Penyewaan dan Sewa Guna Usaha Tanpa Hak Opsi, Ketenagakerjaan, Agen Perjalanan dan Penunjang Usaha Lainnya"
    elif predict == 1:
        label = "P"
        desc = "Pendidikan"
    elif predict == 2:
        label = "K"
        desc = "Aktivitas Keuangan dan Asuransi"
    
    return {'Label':label,'Descriptions':desc}

    
    
    


In [ ]:
result = getPredict('Guru les')

In [ ]:
result

## Load and Use Model

In [ ]:
from transformers import AutoConfig

In [ ]:
config = AutoConfig.from_pretrained(paths['SAVE_MODEL'])


In [ ]:
config

In [ ]:
new_model = AutoModelForSequenceClassification.from_config(config)


In [ ]:
new_model

In [ ]:
load_model = AutoModelForSequenceClassification.from_pretrained(paths['SAVE_MODEL'])
load_token = AutoTokenizer.from_pretrained(paths['SAVE_MODEL'])


In [ ]:
paths['SAVE_MODEL']


In [ ]:
load_model

In [ ]:
from transformers import pipeline

In [ ]:
classifier = pipeline("sentiment-analysis", model="./model/")

In [ ]:
classifier

In [ ]:
results = classifier("penyewaan wedding")

In [ ]:
results

In [ ]:
task_evaluator = evaluator("sentiment-analysis")


In [ ]:
eval_results = task_evaluator.compute(
    model_or_pipeline=classifier,
    data=test_dataset,
    # label_mapping={"": 0, "POSITIVE": 1}
)


In [ ]:
from evaluate import evaluator


In [ ]:
import evaluate

In [ ]:
eval_results = task_evaluator.compute(
    model_or_pipeline="./model/",
    data=test_dataset,
    metric=evaluate.combine(["accuracy", "recall", "precision", "f1"]),
    label_mapping={"LABEL_0": 0,
                   "LABEL_1": 1,
                   "LABEL_2": 2}
)
